# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
gmaps.configure(api_key=g_key)

In [3]:

df = pd.read_csv("output_data/cities.csv")
df.head()

,city_id,city,latitude,longitude,weather,temperature,humidity,pressure,wind_speed,cloudiness,country
0,0,cap malheureux,-19.985995,57.619612,Clouds,72.45,81,1016,3.69,17,MU
1,1,arraial do cabo,-22.966284,-42.024427,Clouds,75.09,88,1015,11.50,75,BR
2,2,puerto ayora,-0.747167,-90.313420,Clouds,74.19,94,1012,3.00,28,EC
3,3,gola gokarannath,28.074659,80.472064,Clear,48.90,62,1020,3.65,0,IN
4,4,hermanus,-34.417500,19.236111,Clear,69.84,85,1012,15.82,4,ZA


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# get data
data = df.loc[:, ["latitude", "longitude"]]

# Create BASE, init the graph
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '0px',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type='SATELLITE')

# init the overlay layers
heat = gmaps.heatmap_layer(data, weights=df.humidity)
fig.add_layer(heat)



# show the plot
fig

Figure(layout=FigureLayout(border='0px', height='600px', margin='0 auto 0 auto', padding='1px', width='800px')…

In [5]:
city_df = df.loc[(df["temperature"] < 80) & (df["temperature"] > 70) \
                                    & (df["wind_speed"] < 10) \
                                    & (df["cloudiness"] == 0)].dropna()
city_df

,city_id,city,latitude,longitude,weather,temperature,humidity,pressure,wind_speed,cloudiness,country
61,61,vicuna,-30.034038,-70.712668,Clear,71.91,56,1017,1.19,0,CL
178,178,shar,24.344558,56.742477,Clear,71.37,62,1021,8.25,0,OM
249,249,calamar,10.252612,-74.914669,Clear,76.08,87,1012,4.92,0,CO
252,252,atar,20.518194,-13.054393,Clear,70.52,21,1018,9.13,0,MR
302,302,port macquarie,-31.429950,152.910352,Clear,79.84,56,1012,8.37,0,AU
307,307,pisco,-13.709981,-76.202979,Rain,73.29,64,1015,4.61,0,PE
445,445,am timan,11.039637,20.282643,Clear,70.47,32,1011,6.58,0,TD
559,559,rafaela,-31.252692,-61.491676,Clear,70.99,65,1011,5.01,0,AR


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
hotel_df = city_df[["city","latitude", "longitude"]].copy()
hotel_df["hotel"] = ""
hotel_df

,city,latitude,longitude,hotel
61,vicuna,-30.034038,-70.712668,
178,shar,24.344558,56.742477,
249,calamar,10.252612,-74.914669,
252,atar,20.518194,-13.054393,
302,port macquarie,-31.429950,152.910352,
307,pisco,-13.709981,-76.202979,
445,am timan,11.039637,20.282643,
559,rafaela,-31.252692,-61.491676,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["latitude"]
    lng = row["longitude"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "hotel"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df
    

,city,latitude,longitude,hotel
61,vicuna,-30.034038,-70.712668,Los Mosaicos
178,shar,24.344558,56.742477,Atlas Hotel Apartment
249,calamar,10.252612,-74.914669,Casa principal
252,atar,20.518194,-13.054393,Oumou Elghoura
302,port macquarie,-31.429950,152.910352,Rydges Port Macquarie
307,pisco,-13.709981,-76.202979,Hotel San Isidro Oficial
445,am timan,11.039637,20.282643,Hôtel d'hébergements
559,rafaela,-31.252692,-61.491676,Parra Hotel & Suites


In [8]:
hotel_all =pd.merge(hotel_df, city_df,)
hotel_all


,city,latitude,longitude,hotel,city_id,weather,temperature,humidity,pressure,wind_speed,cloudiness,country
0,vicuna,-30.034038,-70.712668,Los Mosaicos,61,Clear,71.91,56,1017,1.19,0,CL
1,shar,24.344558,56.742477,Atlas Hotel Apartment,178,Clear,71.37,62,1021,8.25,0,OM
2,calamar,10.252612,-74.914669,Casa principal,249,Clear,76.08,87,1012,4.92,0,CO
3,atar,20.518194,-13.054393,Oumou Elghoura,252,Clear,70.52,21,1018,9.13,0,MR
4,port macquarie,-31.429950,152.910352,Rydges Port Macquarie,302,Clear,79.84,56,1012,8.37,0,AU
5,pisco,-13.709981,-76.202979,Hotel San Isidro Oficial,307,Rain,73.29,64,1015,4.61,0,PE
6,am timan,11.039637,20.282643,Hôtel d'hébergements,445,Clear,70.47,32,1011,6.58,0,TD
7,rafaela,-31.252692,-61.491676,Parra Hotel & Suites,559,Clear,70.99,65,1011,5.01,0,AR


In [9]:
infos = []

for indx, row in hotel_all.iterrows():
    info_box_template = f"""<dl>
                        <dt>Country:</dt><dd>{row.country}</dd>
                        <dt>City:</dt><dd>{row.city}</dd>    
                        <dt>Location:</dt><dd>({row.latitude}, {row.longitude})</dd>
                         <dt>Hotel Name:</dt><dd>{row.hotel}</dd>
                        </dl>
                        """
    infos.append(info_box_template)
    
infos[0]

'<dl>\n                        <dt>Country:</dt><dd>CL</dd>\n                        <dt>City:</dt><dd>vicuna</dd>    \n                        <dt>Location:</dt><dd>(-30.0340383, -70.7126682)</dd>\n                         <dt>Hotel Name:</dt><dd>Los Mosaicos</dd>\n                        </dl>\n                        '

In [ ]:
# Add marker layer ontop of heat map


# Display figure


In [10]:
 # get data
data = df.loc[:, ["latitude", "longitude"]]
hotel_data = hotel_all.loc[:, ["latitude", "longitude"]]
# Create BASE, init the graph
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '0px',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type='SATELLITE')

# init the overlay layers
heat = gmaps.heatmap_layer(data, weights=df.humidity)
fig.add_layer(heat)

markers = gmaps.marker_layer(data, info_box_content=info_box_template)
fig.add_layer(markers)

# show the plot
fig 

Figure(layout=FigureLayout(border='0px', height='600px', margin='0 auto 0 auto', padding='1px', width='800px')…